In [2]:
from tool import *
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.styles import Alignment

# 변수 셋팅

In [17]:
ver = 'Latest'
today = datetime.date.today()
frozen = 3
vendor_list = ['Quanta', 'Pegatron']

# 1. GSCP에서 업체별 보정 SP를 엑셀로 만듬

In [3]:
correct_gscp_data(today, frozen, vendor_list, ver)

** 2023-02-20(W08) 기준의 GSCP SP Latest 로 작업합니다. 확정구간 : 3주 **
-- 전체 업체 2 개 중 1 번째, Quanta 의 데이타 작업 시작합니다.--
    1. Quanta의 GSCP raw 데이타 Latest SP를 정리합니다.
    2. Quanta의 선적 결과 DB 에서 이전 주차의 선적값을 SR에 업데이트합니다.
    3. Quanta의 지난 주 기준 Open PO 를 계산하여 이번주 SP의 확정 구간의 PO값에 업데이트 합니다.
    4. Quanta의 조건에 따른 Real SP를 업데이트하고 엑셀 파일로 출력합니다.
-- 전체 업체 2 개 중 2 번째, Pegatron 의 데이타 작업 시작합니다.--
    1. Pegatron의 GSCP raw 데이타 Latest SP를 정리합니다.
    2. Pegatron의 선적 결과 DB 에서 이전 주차의 선적값을 SR에 업데이트합니다.
    3. Pegatron의 지난 주 기준 Open PO 를 계산하여 이번주 SP의 확정 구간의 PO값에 업데이트 합니다.
    4. Pegatron의 조건에 따른 Real SP를 업데이트하고 엑셀 파일로 출력합니다.


In [4]:
fn = get_filename()
wb = load_workbook(fn)

color = PatternFill(fgColor='BFBFBF', fill_type='solid')

# 첫번째 Sheet(SP Analysis)
sh = wb[wb.sheetnames[0]]
for cell in sh[1]:
    cell.fill = color
    cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
sh.column_dimensions['B'].width = 20
sh.freeze_panes = 'E2'

# 두번째 Sheet(Abnormal SP list)
sh = wb[wb.sheetnames[1]]
for cell in sh[1]:
    cell.fill = color
    cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
sh.column_dimensions['B'].width = 20
sh.column_dimensions['C'].width = 20
sh.freeze_panes = 'F2'

sh_names = wb.sheetnames
for sh in sh_names:
    wb[sh].sheet_view.showGridLines = False

wb.save(fn)

# 2. 보정된 SP가 있는 Excel File을 불러와 DB에 저장

In [3]:
with open('D:/Data/GSCP raw data.bin', 'rb') as f:
    gscp_df = pickle.load(f)

gscp_df.pivot_table('To Site', index='From Site', columns=['Ref', 'Ver'], aggfunc='count')

Ref       2022-09-05(W36)             2022-09-12(W37)                     \
Ver                 Final ODM Release           Final Latest ODM Release   
From Site                                                                  
Pegatron            152.0        30.0           151.0   53.0        31.0   
Quanta              375.0       143.0           375.0  170.0       144.0   
Wanlida              19.0         4.0            19.0   10.0         4.0   

Ref       2022-09-19(W38)                    2022-09-26(W39)         ...  \
Ver                 Final Latest ODM Release           Final Latest  ...   
From Site                                                            ...   
Pegatron            149.0   53.0        31.0           149.0   53.0  ...   
Quanta              373.0  174.0       144.0           373.0  171.0  ...   
Wanlida              19.0   10.0         3.0            15.0    7.0  ...   

Ref       2023-01-30(W05)             2023-02-06(W06)                     \
Ver                Latest ODM Release           Final Latest ODM Release   
From Site                                                                  
Pegatron             84.0        17.0           113.0  180.0        19.0   
Quanta              446.0       148.0           402.0  880.0       147.0   
Wanlida               NaN         NaN             NaN    NaN         NaN   

Ref       2023-02-13(W07)                    2023-02-20(W08)         
Ver                 Final Latest ODM Release           Final Latest  
From Site                                                            
Pegatron            113.0  174.0        19.0           113.0  138.0  
Quanta              398.0  870.0       145.0           396.0  725.0  
Wanlida               NaN    NaN         NaN             NaN    NaN  

[3 rows x 72 columns]

In [5]:
gscp_df['Ref'].unique()[-1]

'2023-02-20(W08)'

In [42]:
gscp_df[gscp_df['Ref']==gscp_df['Ref'].unique()[-1]]['Updated_at'].unique()[-1]

'23022320'

In [43]:
gscp_df[(gscp_df['From Site']=='Quanta') & (gscp_df['Ref']==gscp_df['Ref'].unique()[-1]) & (gscp_df['Updated_at']=='23022320')]['2023-02-20(W08)'].sum()

3116

In [22]:
c1 = sp['From Site']=='Quanta'
sp[c1]['2023-02-20(W08)'].sum()

3116

In [35]:
fname = get_filename()
update_time = fname.split('_')[-2].split('.')[0]

In [36]:
sp = pd.read_excel(fname, sheet_name='abnormal SP list')

sp = sp[sp['Category'] == 'Real_SP']
sp = sp.drop(columns=['Type', 'Category'])
sp = sp.reset_index(drop=True)
sp.insert(0, 'Ref', get_weekname(today))
sp.insert(1, 'Frozen', frozen)
sp.insert(1, 'Updated_at', update_time)
sp.insert(1, 'Ver', ver)
sp

,Ref,Ver,Updated_at,Frozen,From Site,Mapping Model.Suffix,To Site,2023-01-02(W01),2023-01-09(W02),2023-01-16(W03),...,2023-06-12(W24),2023-06-19(W25),2023-06-26(W26),2023-07-03(W27),2023-07-10(W28),2023-07-17(W29),2023-07-24(W30),2023-07-31(W31),2023-08-07(W32),2023-08-14(W33)
0,2023-02-20(W08),Latest,23022320,3,Quanta,11TG50Q-C.AC10KN,EKHQ,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2023-02-20(W08),Latest,23022320,3,Quanta,11TG50Q-E.AC10KN,EKHQ,54,0,0,...,30,30,30,0,30,0,30,0,0,0
2,2023-02-20(W08),Latest,23022320,3,Quanta,14T90P-G.ARTGK,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2023-02-20(W08),Latest,23022320,3,Quanta,14T90Q-G.AA75A3,EASL,0,15,0,...,0,0,0,0,0,0,0,0,0,0
4,2023-02-20(W08),Latest,23022320,3,Quanta,14T90Q-G.AA78B,EEES,0,40,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,2023-02-20(W08),Latest,23022320,3,Pegatron,CL600N-6N.AEUQ,EEDG,0,0,250,...,0,0,0,0,0,0,0,0,0,0
169,2023-02-20(W08),Latest,23022320,3,Pegatron,CL600N-6N.AEUQ,EEFS,0,0,0,...,0,0,0,0,0,0,0,0,0,0
170,2023-02-20(W08),Latest,23022320,3,Pegatron,CL600N-6Q.AKRQ,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
171,2023-02-20(W08),Latest,23022320,3,Pegatron,CL600W-AP.AUBQ,ENCI,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
gscp_df = pd.concat([gscp_df, sp])
gscp_df = gscp_df.fillna(0)
gscp_df = gscp_df.convert_dtypes()
gscp_df = gscp_df.reset_index(drop=True)
gscp_df

,Ref,Ver,Updated_at,Frozen,From Site,Mapping Model.Suffix,To Site,2022-08-01(W31),2022-08-08(W32),2022-08-15(W33),...,2023-12-11(W50),2023-12-18(W51),2023-12-25(W52),2024-01-01(W01),2024-01-08(W02),2024-01-15(W03),2024-01-22(W04),2024-01-29(W05),2024-02-05(W06),2024-02-12(W07)
0,2022-09-26(W39),Final,22092716,4,Quanta,11TC50Q-E.AC10K,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2022-09-26(W39),Final,22092716,4,Quanta,11TC50Q-E.AC10ML,EKHQ,4,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2022-09-26(W39),Final,22092716,4,Quanta,11TG50Q-E.AC10KN,EKHQ,15,231,63,...,0,0,0,0,0,0,0,0,0,0
3,2022-09-26(W39),Final,22092716,4,Quanta,14T90P-G.AA54A3,EASL,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2022-09-26(W39),Final,22092716,4,Quanta,14T90P-G.AA54C,ECNT,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30243,2023-02-20(W08),Latest,23022320,3,Pegatron,CL600N-6N.AEUQ,EEDG,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30244,2023-02-20(W08),Latest,23022320,3,Pegatron,CL600N-6N.AEUQ,EEFS,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30245,2023-02-20(W08),Latest,23022320,3,Pegatron,CL600N-6Q.AKRQ,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30246,2023-02-20(W08),Latest,23022320,3,Pegatron,CL600W-AP.AUBQ,ENCI,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 6. GSCP 데이타에 매주차별로 업체별 data가 있는 지 확인용

In [39]:
gscp_df.pivot_table('To Site', index='From Site', columns=['Ref', 'Ver'], aggfunc='count')

Ref       2022-09-05(W36)             2022-09-12(W37)                     \
Ver                 Final ODM Release           Final Latest ODM Release   
From Site                                                                  
Pegatron            152.0        30.0           151.0   53.0        31.0   
Quanta              375.0       143.0           375.0  170.0       144.0   
Wanlida              19.0         4.0            19.0   10.0         4.0   

Ref       2022-09-19(W38)                    2022-09-26(W39)         ...  \
Ver                 Final Latest ODM Release           Final Latest  ...   
From Site                                                            ...   
Pegatron            149.0   53.0        31.0           149.0   53.0  ...   
Quanta              373.0  174.0       144.0           373.0  171.0  ...   
Wanlida              19.0   10.0         3.0            15.0    7.0  ...   

Ref       2023-01-30(W05)             2023-02-06(W06)                     \
Ver                Latest ODM Release           Final Latest ODM Release   
From Site                                                                  
Pegatron             84.0        17.0           113.0  180.0        19.0   
Quanta              446.0       148.0           402.0  880.0       147.0   
Wanlida               NaN         NaN             NaN    NaN         NaN   

Ref       2023-02-13(W07)                    2023-02-20(W08)         
Ver                 Final Latest ODM Release           Final Latest  
From Site                                                            
Pegatron            113.0  174.0        19.0           113.0  138.0  
Quanta              398.0  870.0       145.0           396.0  725.0  
Wanlida               NaN    NaN         NaN             NaN    NaN  

[3 rows x 72 columns]

# 7. 최종 업데이트된 데이타 저장

In [40]:
with open('D:Data/GSCP raw data.bin', 'wb') as f:
    pickle.dump(gscp_df, f)

In [41]:
# DB 백업용 폴더에도 저장
with open('D:/Data/DB backup/GSCP raw data.bin', 'wb') as f:
    pickle.dump(gscp_df, f)

## ODM 으로 발송한 SP를 GSCP DB에 업데이트

### 1) Quanta, Pegatron의 물동일 경우

In [17]:
frozen = 3 # 확정구간 설정

In [24]:
# 23년 1월 이후에는 아래로 사용
fname = get_filename()
tab_name = f"SP_{fname.split('_')[-1].split('.')[0][:-1]}"
thisweek = get_weekname(datetime.date(2023, int(tab_name[3:5]), int(tab_name[5:7])))
ver = 'ODM Release'
updated_time = (datetime.date(2023, int(tab_name[3:5]), int(tab_name[5:7])) + datetime.timedelta(2)).strftime('%y%m%d') + '00'

vendor = fname.split('_')[2].split(' ')[0]

sp = get_PC_ODM_shipment_plan(fname, vendor.upper(), tab_name).groupby(['Mapping Model.Suffix', 'To Site']).sum()
sp = sp.loc[:, get_previous_weeklist(thisweek)[0]:]
sp['Sum'] = sp.sum(axis=1)
sp = sp[sp['Sum'] > 0]
sp = sp.drop(columns='Sum')
sp = sp.reset_index()
sp.insert(0, 'Ref', thisweek)
sp.insert(1, 'Frozen', frozen)
sp.insert(1, 'Updated_at', updated_time)
sp.insert(1, 'Ver', ver)
sp.insert(2, 'From Site', vendor)
sp

,Ref,Ver,From Site,Updated_at,Frozen,Mapping Model.Suffix,To Site,2023-01-02(W01),2023-01-09(W02),2023-01-16(W03),...,2023-07-24(W30),2023-07-31(W31),2023-08-07(W32),2023-08-14(W33),2023-08-21(W34),2023-08-28(W35),2023-09-04(W36),2023-09-11(W37),2023-09-18(W38),2023-09-25(W39)
0,2023-02-13(W07),ODM Release,Pegatron,23021500,3,13U70P-G.AAX7U1,ENUS_V,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2023-02-13(W07),ODM Release,Pegatron,23021500,3,13U70Q-G.AA74J,EJJP,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2023-02-13(W07),ODM Release,Pegatron,23021500,3,13U70Q-G.AR31J,EJJP,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2023-02-13(W07),ODM Release,Pegatron,23021500,3,13U70Q-G.AR55B,EEES,30,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2023-02-13(W07),ODM Release,Pegatron,23021500,3,14TG30Q-E.A710KN,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2023-02-13(W07),ODM Release,Pegatron,23021500,3,14TN30Q-E.AO14KN,EKHQ,870,0,0,...,0,0,0,0,0,0,0,0,0,0
6,2023-02-13(W07),ODM Release,Pegatron,23021500,3,14U30P-E.A716K,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,2023-02-13(W07),ODM Release,Pegatron,23021500,3,14U30P-E.A726K,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,2023-02-13(W07),ODM Release,Pegatron,23021500,3,17U70Q-G.AA5NL,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,2023-02-13(W07),ODM Release,Pegatron,23021500,3,17U70Q-G.ARLGL,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
# 23년 1월 shipment plan에 전월이 없어 임시로 사용
fname = get_filename()
tab_name = f"SP_{fname.split('_')[-1].split('.')[0][:-1]}"
thisweek = get_weekname(datetime.date(2023, int(tab_name[3:5]), int(tab_name[5:7])))
ver = 'ODM Release'
updated_time = f'22{tab_name[3:5]}{format(int(tab_name[5:7])+2, "02")}00'

vendor = fname.split('_')[2].split(' ')[0]
fname, thisweek, vendor

('D:/Shipment Plan/ODM shipment plan/2301_W04_Quanta FCST_0123W.xlsb',
 '2023-01-23(W04)',
 'Quanta')

In [26]:
sp = get_PC_ODM_shipment_plan(fname, vendor.upper(), tab_name).groupby(['Mapping Model.Suffix', 'To Site']).sum()
sp = sp.loc[:, get_weekname(datetime.date(2023, 1,5)):]
sp = sp.reset_index()
sp.insert(0, 'Ref', thisweek)
sp.insert(1, 'Frozen', frozen)
sp.insert(1, 'Updated_at', updated_time)
sp.insert(1, 'Ver', ver)
sp.insert(2, 'From Site', vendor)
sp

,Ref,Ver,From Site,Updated_at,Frozen,Mapping Model.Suffix,To Site,2023-01-02(W01),2023-01-09(W02),2023-01-16(W03),...,2023-07-24(W30),2023-07-31(W31),2023-08-07(W32),2023-08-14(W33),2023-08-21(W34),2023-08-28(W35),2023-09-04(W36),2023-09-11(W37),2023-09-18(W38),2023-09-25(W39)
0,2023-01-23(W04),ODM Release,Quanta,22012500,3,11TG50Q-C.AC10KN,EKHQ,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2023-01-23(W04),ODM Release,Quanta,22012500,3,11TG50Q-E.AC10KN,EKHQ,54,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2023-01-23(W04),ODM Release,Quanta,22012500,3,14T90P-G.ARTGK,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2023-01-23(W04),ODM Release,Quanta,22012500,3,14T90Q-G.AA75A3,EASL,0,15,0,...,0,0,0,0,0,0,0,0,0,0
4,2023-01-23(W04),ODM Release,Quanta,22012500,3,14T90Q-G.AA78B,EEES,0,40,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,2023-01-23(W04),ODM Release,Quanta,22012500,3,16TD90Q-G.AX70K,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
142,2023-01-23(W04),ODM Release,Quanta,22012500,3,16TD90R-G.AX56K,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
143,2023-01-23(W04),ODM Release,Quanta,22012500,3,16TD90R-G.AX76K,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
144,2023-01-23(W04),ODM Release,Quanta,22012500,3,16TG90Q-G.AA75KN,EKHQ,0,0,54,...,0,0,0,0,0,0,0,0,0,0


In [25]:
with open('D:/Data/GSCP raw data.bin', 'rb') as f:
    gscp_df = pickle.load(f)

gscp_df.pivot_table('To Site', index='From Site', columns=['Ref', 'Ver'], aggfunc='count')

Ref       2022-09-05(W36)             2022-09-12(W37)                     \
Ver                 Final ODM Release           Final Latest ODM Release   
From Site                                                                  
Pegatron            152.0        30.0           151.0   53.0        31.0   
Quanta              375.0       143.0           375.0  170.0       144.0   
Wanlida              19.0         4.0            19.0   10.0         4.0   

Ref       2022-09-19(W38)                    2022-09-26(W39)         ...  \
Ver                 Final Latest ODM Release           Final Latest  ...   
From Site                                                            ...   
Pegatron            149.0   53.0        31.0           149.0   53.0  ...   
Quanta              373.0  174.0       144.0           373.0  171.0  ...   
Wanlida              19.0   10.0         3.0            15.0    7.0  ...   

Ref       2023-01-23(W04) 2023-01-30(W05)                    2023-02-06(W06)  \
Ver           ODM Release           Final Latest ODM Release           Final   
From Site                                                                      
Pegatron             18.0           112.0   84.0        17.0           113.0   
Quanta              146.0           401.0  446.0       148.0           402.0   
Wanlida               NaN             NaN    NaN         NaN             NaN   

Ref                          2023-02-13(W07)                     
Ver       Latest ODM Release           Final Latest ODM Release  
From Site                                                        
Pegatron   180.0        19.0           113.0  174.0         NaN  
Quanta     880.0       147.0           398.0  870.0       145.0  
Wanlida      NaN         NaN             NaN    NaN         NaN  

[3 rows x 70 columns]

In [26]:
gscp_df = pd.concat([gscp_df, sp])
gscp_df = gscp_df.fillna(0)
gscp_df = gscp_df.convert_dtypes()
gscp_df = gscp_df.reset_index(drop=True)
gscp_df

,Ref,Ver,Updated_at,Frozen,From Site,Mapping Model.Suffix,To Site,2022-08-01(W31),2022-08-08(W32),2022-08-15(W33),...,2023-12-04(W49),2023-12-11(W50),2023-12-18(W51),2023-12-25(W52),2024-01-01(W01),2024-01-08(W02),2024-01-15(W03),2024-01-22(W04),2024-01-29(W05),2024-02-05(W06)
0,2022-09-26(W39),Final,22092716,4,Quanta,11TC50Q-E.AC10K,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2022-09-26(W39),Final,22092716,4,Quanta,11TC50Q-E.AC10ML,EKHQ,4,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2022-09-26(W39),Final,22092716,4,Quanta,11TG50Q-E.AC10KN,EKHQ,15,231,63,...,0,0,0,0,0,0,0,0,0,0
3,2022-09-26(W39),Final,22092716,4,Quanta,14T90P-G.AA54A3,EASL,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2022-09-26(W39),Final,22092716,4,Quanta,14T90P-G.AA54C,ECNT,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28871,2023-02-13(W07),ODM Release,23021500,3,Pegatron,17U70Q-P.AP7GL,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28872,2023-02-13(W07),ODM Release,23021500,3,Pegatron,17U70Q-P.AS7CL,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28873,2023-02-13(W07),ODM Release,23021500,3,Pegatron,17UD70P-P.AX76K,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28874,2023-02-13(W07),ODM Release,23021500,3,Pegatron,17UD70Q-P.AX70K,EKHQ,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 6. GSCP 데이타에 매주차별로 업체별 data가 있는 지 확인용

In [27]:
gscp_df.pivot_table('To Site', index='From Site', columns=['Ref', 'Ver'], aggfunc='count')

Ref       2022-09-05(W36)             2022-09-12(W37)                     \
Ver                 Final ODM Release           Final Latest ODM Release   
From Site                                                                  
Pegatron            152.0        30.0           151.0   53.0        31.0   
Quanta              375.0       143.0           375.0  170.0       144.0   
Wanlida              19.0         4.0            19.0   10.0         4.0   

Ref       2022-09-19(W38)                    2022-09-26(W39)         ...  \
Ver                 Final Latest ODM Release           Final Latest  ...   
From Site                                                            ...   
Pegatron            149.0   53.0        31.0           149.0   53.0  ...   
Quanta              373.0  174.0       144.0           373.0  171.0  ...   
Wanlida              19.0   10.0         3.0            15.0    7.0  ...   

Ref       2023-01-23(W04) 2023-01-30(W05)                    2023-02-06(W06)  \
Ver           ODM Release           Final Latest ODM Release           Final   
From Site                                                                      
Pegatron             18.0           112.0   84.0        17.0           113.0   
Quanta              146.0           401.0  446.0       148.0           402.0   
Wanlida               NaN             NaN    NaN         NaN             NaN   

Ref                          2023-02-13(W07)                     
Ver       Latest ODM Release           Final Latest ODM Release  
From Site                                                        
Pegatron   180.0        19.0           113.0  174.0        19.0  
Quanta     880.0       147.0           398.0  870.0       145.0  
Wanlida      NaN         NaN             NaN    NaN         NaN  

[3 rows x 70 columns]

# 7. 최종 업데이트된 데이타 저장

In [28]:
with open('D:/Data/GSCP raw data.bin', 'wb') as f:
    pickle.dump(gscp_df, f)

In [29]:
# DB 백업용 폴더에도 저장
with open('D:/Data/DB backup/GSCP raw data.bin', 'wb') as f:
    pickle.dump(gscp_df, f)

### 2) Wanlida 의 물동일 경우

In [28]:
fname = get_filename()
vendor = fname.split('/')[1]
thisweek = get_weekname(datetime.date.fromisoformat(f"{fname.split('_')[2].split('/')[1][:4]}-{fname.split('_')[2].split('/')[1][4:6]}-{fname.split('_')[2].split('/')[1][6:]}"))
updated_time = f"{fname.split('_')[2].split('/')[1][2:]}00"
ver = 'ODM Release'
ref = thisweek
frozen = 3

sp = pd.read_excel(fname, sheet_name=thisweek, skiprows=1).groupby(['Mapping Model.Suffix', 'To Site']).sum()
sp['Sum'] = sp.sum(axis=1)
sp = sp[sp['Sum'] > 0]
sp = sp.drop(columns='Sum')
sp = sp.reset_index()
sp.insert(0, 'Ref', thisweek)
sp.insert(1, 'Frozen', frozen)
sp.insert(1, 'Updated_at', updated_time)
sp.insert(1, 'Ver', ver)
sp.insert(2, 'From Site', vendor)
sp

,Ref,Ver,From Site,Updated_at,Frozen,Mapping Model.Suffix,To Site,2022-10-31(W44),2022-11-07(W45),2022-11-14(W46),2022-11-21(W47),2022-11-28(W48),2022-12-05(W49)
0,2022-12-05(W49),ODM Release,Wanlida,22120800,3,PH30N.AEUQ,EEPT,0,0,0,100,0,0
1,2022-12-05(W49),ODM Release,Wanlida,22120800,3,PH30N.AMIQ,EMSB_EMAM_DO,0,0,0,470,0,30
2,2022-12-05(W49),ODM Release,Wanlida,22120800,3,PH30N.AMIQ,EMSB_EXTRA_DO,0,0,0,500,0,0


In [62]:
df = pd.read_excel('D:/Shipment Plan/GSCP Data/W5_GSCP_SP_correction__Quanta_Pegatron_23020109_Latest.xlsx', sheet_name='SP Analysis')
df

,From Site,Mapping Model.Suffix,To Site,Category,2023-01-02(W01),2023-01-09(W02),2023-01-16(W03),2023-01-23(W04),2023-01-30(W05),2023-02-06(W06),...,2023-05-22(W21),2023-05-29(W22),2023-06-05(W23),2023-06-12(W24),2023-06-19(W25),2023-06-26(W26),2023-07-03(W27),2023-07-10(W28),2023-07-17(W29),2023-07-24(W30)
0,Quanta,11TG50Q-C.AC10KN,EKHQ,SP,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Quanta,11TG50Q-C.AC10KN,EKHQ,SR,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Quanta,11TG50Q-C.AC10KN,EKHQ,PO,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Quanta,11TG50Q-C.AC10KN,EKHQ,Real_SP,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Quanta,11TG50Q-E.AC10KN,EKHQ,SP,54,0,0,0,780,0,...,30,30,30,0,40,30,30,0,30,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703,Pegatron,CL600W-AP.AUBQ,ENCI,Real_SP,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
704,Pegatron,CL600W-AQ.AKRQ,EKHQ,SP,0,0,80,0,100,0,...,0,0,0,0,0,0,0,0,0,0
705,Pegatron,CL600W-AQ.AKRQ,EKHQ,SR,0,0,80,0,0,0,...,0,0,0,0,0,0,0,0,0,0
706,Pegatron,CL600W-AQ.AKRQ,EKHQ,PO,0,0,0,0,20,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
df['Series'] = df['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).replace(srt_model)
c1 = df['From Site'] == 'Quanta'
df[c1]

,From Site,Mapping Model.Suffix,To Site,Category,2023-01-02(W01),2023-01-09(W02),2023-01-16(W03),2023-01-23(W04),2023-01-30(W05),2023-02-06(W06),...,2023-05-29(W22),2023-06-05(W23),2023-06-12(W24),2023-06-19(W25),2023-06-26(W26),2023-07-03(W27),2023-07-10(W28),2023-07-17(W29),2023-07-24(W30),Series
0,Quanta,11TG50Q-C.AC10KN,EKHQ,SP,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11T50Q
1,Quanta,11TG50Q-C.AC10KN,EKHQ,SR,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11T50Q
2,Quanta,11TG50Q-C.AC10KN,EKHQ,PO,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11T50Q
3,Quanta,11TG50Q-C.AC10KN,EKHQ,Real_SP,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11T50Q
4,Quanta,11TG50Q-E.AC10KN,EKHQ,SP,54,0,0,0,780,0,...,30,30,0,40,30,30,0,30,0,11T50Q
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,Quanta,16TG90Q-G.AA75KN,EKHQ,Real_SP,0,0,54,18,88,0,...,0,0,0,0,0,0,0,0,0,16T90Q
592,Quanta,16TG90Q-G.AR55KN,EKHQ,SP,0,0,54,36,110,0,...,0,0,0,0,0,0,0,0,0,16T90Q
593,Quanta,16TG90Q-G.AR55KN,EKHQ,SR,0,0,54,36,0,0,...,0,0,0,0,0,0,0,0,0,16T90Q
594,Quanta,16TG90Q-G.AR55KN,EKHQ,PO,0,0,0,0,110,0,...,0,0,0,0,0,0,0,0,0,16T90Q


In [77]:
c2 = df['Category'].isin(['Real_SP', 'PO'])

In [79]:
df[c1& c2].pivot_table(get_weeklist(df), index=['Series', 'Category'], aggfunc=sum).to_clipboard()